# Module 1.4: 幾何變換 (Geometric Transforms)

## 學習目標

本 notebook 將帶你實作影像的幾何變換：

1. **影像旋轉**：理解旋轉矩陣
2. **最近鄰插值 (Nearest Neighbor)**：最簡單的插值方法
3. **雙線性插值 (Bilinear Interpolation)**：更平滑的插值
4. **仿射變換 (Affine Transform)**：通用的 2D 變換

## 前置要求
- 完成 Module 0（向量與矩陣）
- 完成 01_convolution.ipynb

## 參考資料
- Gonzalez & Woods, Digital Image Processing, Ch. 2.6

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import sys
sys.path.append('../../')

try:
    from utils.image_io import load_grayscale, save_image
except:
    def load_grayscale(path):
        return np.array(Image.open(path).convert('L'), dtype=np.float64)
    def save_image(arr, path):
        Image.fromarray(arr.astype(np.uint8)).save(path)

print("環境設定完成！")

## 建立測試影像

建立一個具有明顯特徵的測試影像，方便觀察變換效果。

In [ ]:
def create_test_image(size=100):
    """
    建立一個具有明顯特徵的測試影像。
    包含：方塊、圓形、三角形、文字
    """
    image = np.zeros((size, size), dtype=np.float64)
    
    # 背景漸層
    for i in range(size):
        image[i, :] = 30 + i * 0.5
    
    # 左上：白色方塊
    image[10:30, 10:30] = 255
    
    # 右上：圓形
    y, x = np.ogrid[:size, :size]
    center_y, center_x = 20, 70
    radius = 12
    mask = (x - center_x)**2 + (y - center_y)**2 <= radius**2
    image[mask] = 200
    
    # 左下：三角形
    for i in range(20):
        image[70+i, 20-i:20+i] = 180
    
    # 右下：棋盤格
    for i in range(5):
        for j in range(5):
            if (i + j) % 2 == 0:
                image[60+i*8:60+(i+1)*8, 60+j*8:60+(j+1)*8] = 255
    
    # 十字準星（幫助觀察旋轉中心）
    center = size // 2
    image[center-1:center+2, center-15:center+15] = 150
    image[center-15:center+15, center-1:center+2] = 150
    
    return image

# 建立並顯示測試影像
test_image = create_test_image(100)

plt.figure(figsize=(6, 6))
plt.imshow(test_image, cmap='gray', vmin=0, vmax=255)
plt.title('Test Image')
plt.colorbar()
plt.show()

print(f"影像大小：{test_image.shape}")

---

# Part 1: 旋轉矩陣 (Rotation Matrix)

## 數學原理

2D 旋轉可以用矩陣表示。對於一個點 $(x, y)$，繞原點旋轉角度 $\theta$：

$$
\begin{bmatrix} x' \\ y' \end{bmatrix} = 
\begin{bmatrix} 
\cos\theta & -\sin\theta \\
\sin\theta & \cos\theta 
\end{bmatrix}
\begin{bmatrix} x \\ y \end{bmatrix}
$$

### 旋轉的方向
- **正角度 (θ > 0)**：逆時針旋轉
- **負角度 (θ < 0)**：順時針旋轉

### 繞任意點旋轉

如果要繞點 $(c_x, c_y)$ 旋轉，需要：
1. 先平移使旋轉中心到原點
2. 旋轉
3. 再平移回來

$$
\begin{bmatrix} x' \\ y' \end{bmatrix} = 
R(\theta) \cdot \begin{bmatrix} x - c_x \\ y - c_y \end{bmatrix} + 
\begin{bmatrix} c_x \\ c_y \end{bmatrix}
$$

In [ ]:
def rotation_matrix(angle_degrees):
    """
    建立 2D 旋轉矩陣。
    
    Parameters
    ----------
    angle_degrees : float
        旋轉角度（度），正值為逆時針
    
    Returns
    -------
    R : np.ndarray, shape (2, 2)
        旋轉矩陣
    """
    # 轉換為弧度
    theta = np.radians(angle_degrees)
    
    # 建立旋轉矩陣
    cos_t = np.cos(theta)
    sin_t = np.sin(theta)
    
    R = np.array([
        [cos_t, -sin_t],
        [sin_t,  cos_t]
    ])
    
    return R

# 測試旋轉矩陣
print("旋轉 90 度的矩陣：")
R90 = rotation_matrix(90)
print(R90)
print()

# 驗證：旋轉 (1, 0) 90 度應該得到 (0, 1)
point = np.array([1, 0])
rotated = R90 @ point
print(f"原始點：{point}")
print(f"旋轉 90° 後：{rotated}")
print(f"預期結果：[0, 1]")

In [ ]:
# 視覺化旋轉效果
fig, ax = plt.subplots(1, 1, figsize=(8, 8))

# 原始的正方形
square = np.array([
    [0, 0],
    [1, 0],
    [1, 1],
    [0, 1],
    [0, 0]  # 回到起點形成封閉圖形
]).T  # shape (2, 5)

# 繪製原始正方形
ax.plot(square[0], square[1], 'b-', linewidth=2, label='Original')

# 繪製不同角度的旋轉
colors = ['red', 'green', 'purple', 'orange']
angles = [30, 45, 60, 90]

for angle, color in zip(angles, colors):
    R = rotation_matrix(angle)
    rotated = R @ square
    ax.plot(rotated[0], rotated[1], '-', color=color, linewidth=2, label=f'{angle}°')

ax.axhline(0, color='gray', linestyle='--', alpha=0.5)
ax.axvline(0, color='gray', linestyle='--', alpha=0.5)
ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-0.5, 1.5)
ax.set_aspect('equal')
ax.legend()
ax.set_title('Rotation of a Square')
ax.grid(True, alpha=0.3)
plt.show()

---

# Part 2: 影像旋轉的挑戰

## Forward Mapping vs Backward Mapping

### Forward Mapping（正向映射）
```
對於每個原始像素 (x, y):
    計算變換後的位置 (x', y')
    將像素值放到輸出影像的 (x', y') 位置
```

**問題**：
1. (x', y') 可能不是整數座標
2. 輸出影像可能有「空洞」（沒有被填滿的像素）

### Backward Mapping（反向映射）✓
```
對於每個輸出像素 (x', y'):
    計算它在原始影像中對應的位置 (x, y)
    使用插值法取得該位置的像素值
```

**優點**：
1. 輸出影像的每個像素都會被處理
2. 不會有空洞

### 旋轉的反向變換

如果正向變換是 $R(\theta)$，反向變換就是 $R(-\theta) = R(\theta)^T$

In [ ]:
# 示範 Forward vs Backward mapping

def forward_mapping_demo(image, angle):
    """
    使用 Forward Mapping 旋轉影像（有問題的方法）。
    這是為了示範 forward mapping 的問題。
    """
    H, W = image.shape
    output = np.zeros_like(image) - 1  # -1 表示未填充
    
    R = rotation_matrix(angle)
    center_x, center_y = W / 2, H / 2
    
    for y in range(H):
        for x in range(W):
            # 轉換座標（相對於中心）
            dx = x - center_x
            dy = y - center_y
            
            # 正向旋轉
            new_coords = R @ np.array([dx, dy])
            new_x = new_coords[0] + center_x
            new_y = new_coords[1] + center_y
            
            # 四捨五入到最近的整數座標
            new_x_int = int(round(new_x))
            new_y_int = int(round(new_y))
            
            # 如果在範圍內，填入像素值
            if 0 <= new_x_int < W and 0 <= new_y_int < H:
                output[new_y_int, new_x_int] = image[y, x]
    
    return output

# 示範
forward_result = forward_mapping_demo(test_image, 30)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].imshow(test_image, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Original')

# 把未填充的像素標記為紅色
display = np.zeros((*forward_result.shape, 3))
display[:, :, 0] = np.where(forward_result == -1, 255, forward_result)  # R
display[:, :, 1] = np.where(forward_result == -1, 0, forward_result)    # G
display[:, :, 2] = np.where(forward_result == -1, 0, forward_result)    # B
display = display.astype(np.uint8)

axes[1].imshow(display)
axes[1].set_title('Forward Mapping (紅色=空洞)')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

# 計算空洞數量
holes = np.sum(forward_result == -1)
total = forward_result.size
print(f"空洞像素數：{holes} / {total} ({holes/total*100:.1f}%)")

---

# Part 3: 最近鄰插值 (Nearest Neighbor Interpolation)

## 概念說明

當計算出的座標 $(x, y)$ 不是整數時，**最近鄰插值**直接使用最接近的整數座標的像素值。

$$
I'(x, y) = I(\text{round}(x), \text{round}(y))
$$

### 優點：
- 簡單、快速
- 保持銳利邊緣

### 缺點：
- 可能產生鋸齒（aliasing）
- 放大時效果不好

In [ ]:
def rotate_nearest_neighbor(image, angle_degrees):
    """
    使用最近鄰插值旋轉影像。
    
    Parameters
    ----------
    image : np.ndarray, shape (H, W)
        輸入灰階影像
    angle_degrees : float
        旋轉角度（度），正值為逆時針
    
    Returns
    -------
    rotated : np.ndarray
        旋轉後的影像
    
    實作步驟（Backward Mapping）：
    1. 對於輸出影像的每個像素 (x', y')
    2. 計算它在原始影像中的對應位置 (x, y) = R^(-1) * (x', y')
    3. 使用最近鄰插值取得像素值
    """
    H, W = image.shape
    output = np.zeros_like(image)
    
    # 旋轉中心
    center_x = W / 2
    center_y = H / 2
    
    # 反向旋轉矩陣（角度取負）
    R_inv = rotation_matrix(-angle_degrees)
    
    # 對每個輸出像素
    for y_out in range(H):
        for x_out in range(W):
            # 相對於中心的座標
            dx = x_out - center_x
            dy = y_out - center_y
            
            # 反向變換找到原始座標
            orig = R_inv @ np.array([dx, dy])
            x_orig = orig[0] + center_x
            y_orig = orig[1] + center_y
            
            # 最近鄰插值：四捨五入
            x_nn = int(round(x_orig))
            y_nn = int(round(y_orig))
            
            # 檢查邊界
            if 0 <= x_nn < W and 0 <= y_nn < H:
                output[y_out, x_out] = image[y_nn, x_nn]
            else:
                output[y_out, x_out] = 0  # 超出範圍填 0
    
    return output

# 測試
rotated_30 = rotate_nearest_neighbor(test_image, 30)
rotated_45 = rotate_nearest_neighbor(test_image, 45)
rotated_90 = rotate_nearest_neighbor(test_image, 90)

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

axes[0].imshow(test_image, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Original')

axes[1].imshow(rotated_30, cmap='gray', vmin=0, vmax=255)
axes[1].set_title('Rotated 30° (NN)')

axes[2].imshow(rotated_45, cmap='gray', vmin=0, vmax=255)
axes[2].set_title('Rotated 45° (NN)')

axes[3].imshow(rotated_90, cmap='gray', vmin=0, vmax=255)
axes[3].set_title('Rotated 90° (NN)')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

---

# Part 4: 雙線性插值 (Bilinear Interpolation)

## 概念說明

**雙線性插值** 使用周圍 4 個像素的加權平均，權重由距離決定。

對於非整數座標 $(x, y)$，設：
- $x_0 = \lfloor x \rfloor$, $x_1 = x_0 + 1$
- $y_0 = \lfloor y \rfloor$, $y_1 = y_0 + 1$
- $dx = x - x_0$（水平距離）
- $dy = y - y_0$（垂直距離）

### 插值公式：

1. 先在水平方向插值：
$$
f_1 = (1-dx) \cdot I(x_0, y_0) + dx \cdot I(x_1, y_0)
$$
$$
f_2 = (1-dx) \cdot I(x_0, y_1) + dx \cdot I(x_1, y_1)
$$

2. 再在垂直方向插值：
$$
f = (1-dy) \cdot f_1 + dy \cdot f_2
$$

### 展開成單一公式：

$$
f(x, y) = (1-dx)(1-dy) \cdot I(x_0, y_0) + dx(1-dy) \cdot I(x_1, y_0) + (1-dx)dy \cdot I(x_0, y_1) + dx \cdot dy \cdot I(x_1, y_1)
$$

In [ ]:
def bilinear_interpolate(image, x, y):
    """
    在影像的非整數座標 (x, y) 進行雙線性插值。
    
    Parameters
    ----------
    image : np.ndarray, shape (H, W)
        輸入影像
    x, y : float
        要插值的座標
    
    Returns
    -------
    value : float
        插值後的像素值
    """
    H, W = image.shape
    
    # 計算四個角點的座標
    x0 = int(np.floor(x))
    x1 = x0 + 1
    y0 = int(np.floor(y))
    y1 = y0 + 1
    
    # 計算距離（權重）
    dx = x - x0
    dy = y - y0
    
    # 處理邊界情況
    x0 = np.clip(x0, 0, W - 1)
    x1 = np.clip(x1, 0, W - 1)
    y0 = np.clip(y0, 0, H - 1)
    y1 = np.clip(y1, 0, H - 1)
    
    # 取得四個角點的像素值
    I00 = image[y0, x0]  # 左上
    I10 = image[y0, x1]  # 右上
    I01 = image[y1, x0]  # 左下
    I11 = image[y1, x1]  # 右下
    
    # 雙線性插值
    value = (
        (1 - dx) * (1 - dy) * I00 +
        dx * (1 - dy) * I10 +
        (1 - dx) * dy * I01 +
        dx * dy * I11
    )
    
    return value


# 視覺化雙線性插值
fig, ax = plt.subplots(1, 1, figsize=(8, 8))

# 建立一個小的 3x3 影像
small_img = np.array([
    [10, 50, 90],
    [30, 100, 60],
    [80, 40, 20]
], dtype=np.float64)

# 繪製原始像素
for i in range(3):
    for j in range(3):
        ax.scatter(j, i, s=500, c=[small_img[i, j]/100], cmap='gray', 
                   vmin=0, vmax=1, edgecolors='black', linewidths=2)
        ax.text(j, i, f'{small_img[i, j]:.0f}', ha='center', va='center', fontsize=10)

# 在 (1.3, 0.7) 進行插值
x_interp = 1.3
y_interp = 0.7
interpolated = bilinear_interpolate(small_img, x_interp, y_interp)

ax.scatter(x_interp, y_interp, s=300, c='red', marker='x', linewidths=3)
ax.text(x_interp + 0.2, y_interp, f'{interpolated:.1f}', color='red', fontsize=12)

# 標記使用的四個角點
ax.plot([1, 2, 2, 1, 1], [0, 0, 1, 1, 0], 'r--', linewidth=2)

ax.set_xlim(-0.5, 2.5)
ax.set_ylim(2.5, -0.5)
ax.set_aspect('equal')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title(f'Bilinear Interpolation at ({x_interp}, {y_interp}) = {interpolated:.1f}')
ax.grid(True, alpha=0.3)
plt.show()

# 驗證計算
print("驗證：")
print(f"四個角點：I(1,0)={small_img[0,1]}, I(2,0)={small_img[0,2]}, I(1,1)={small_img[1,1]}, I(2,1)={small_img[1,2]}")
print(f"dx = 0.3, dy = 0.7")
print(f"插值結果 = (1-0.3)(1-0.7)*50 + 0.3*(1-0.7)*90 + (1-0.3)*0.7*100 + 0.3*0.7*60")
manual = 0.7*0.3*50 + 0.3*0.3*90 + 0.7*0.7*100 + 0.3*0.7*60
print(f"           = {manual:.1f}")

In [ ]:
def rotate_bilinear(image, angle_degrees):
    """
    使用雙線性插值旋轉影像。
    
    Parameters
    ----------
    image : np.ndarray, shape (H, W)
        輸入灰階影像
    angle_degrees : float
        旋轉角度（度），正值為逆時針
    
    Returns
    -------
    rotated : np.ndarray
        旋轉後的影像
    """
    H, W = image.shape
    output = np.zeros_like(image, dtype=np.float64)
    
    center_x = W / 2
    center_y = H / 2
    
    R_inv = rotation_matrix(-angle_degrees)
    
    for y_out in range(H):
        for x_out in range(W):
            dx = x_out - center_x
            dy = y_out - center_y
            
            orig = R_inv @ np.array([dx, dy])
            x_orig = orig[0] + center_x
            y_orig = orig[1] + center_y
            
            # 檢查邊界
            if 0 <= x_orig < W - 1 and 0 <= y_orig < H - 1:
                output[y_out, x_out] = bilinear_interpolate(image, x_orig, y_orig)
            else:
                output[y_out, x_out] = 0
    
    return output

# 比較最近鄰和雙線性
rotated_nn = rotate_nearest_neighbor(test_image, 30)
rotated_bl = rotate_bilinear(test_image, 30)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(test_image, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Original')

axes[1].imshow(rotated_nn, cmap='gray', vmin=0, vmax=255)
axes[1].set_title('Nearest Neighbor')

axes[2].imshow(rotated_bl, cmap='gray', vmin=0, vmax=255)
axes[2].set_title('Bilinear')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# 放大比較細節
# 取一小塊區域來觀察差異
region_nn = rotated_nn[40:60, 40:60]
region_bl = rotated_bl[40:60, 40:60]

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].imshow(region_nn, cmap='gray', vmin=0, vmax=255, interpolation='nearest')
axes[0].set_title('Nearest Neighbor (zoomed)')

axes[1].imshow(region_bl, cmap='gray', vmin=0, vmax=255, interpolation='nearest')
axes[1].set_title('Bilinear (zoomed)')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

print("觀察：")
print("1. Nearest Neighbor 有明顯的鋸齒邊緣")
print("2. Bilinear 的邊緣更平滑")
print("3. 但 Bilinear 的銳利邊緣可能會稍微模糊")

---

# Part 5: 向量化實作（效能優化）

上面的實作使用 for 迴圈，速度較慢。讓我們用向量化的方式重寫。

In [ ]:
def rotate_bilinear_vectorized(image, angle_degrees):
    """
    向量化的雙線性插值旋轉。
    比 for-loop 版本快很多。
    
    Parameters
    ----------
    image : np.ndarray, shape (H, W)
        輸入影像
    angle_degrees : float
        旋轉角度
    
    Returns
    -------
    rotated : np.ndarray
        旋轉後的影像
    """
    H, W = image.shape
    
    # 建立輸出座標網格
    y_out, x_out = np.mgrid[0:H, 0:W]
    
    # 中心
    center_x = W / 2
    center_y = H / 2
    
    # 相對於中心的座標
    dx = x_out - center_x
    dy = y_out - center_y
    
    # 反向旋轉
    theta = np.radians(-angle_degrees)
    cos_t = np.cos(theta)
    sin_t = np.sin(theta)
    
    x_orig = cos_t * dx - sin_t * dy + center_x
    y_orig = sin_t * dx + cos_t * dy + center_y
    
    # 雙線性插值的四個角點座標
    x0 = np.floor(x_orig).astype(int)
    y0 = np.floor(y_orig).astype(int)
    x1 = x0 + 1
    y1 = y0 + 1
    
    # 權重
    wx = x_orig - x0
    wy = y_orig - y0
    
    # 邊界處理
    valid = (x0 >= 0) & (x1 < W) & (y0 >= 0) & (y1 < H)
    
    x0 = np.clip(x0, 0, W - 1)
    x1 = np.clip(x1, 0, W - 1)
    y0 = np.clip(y0, 0, H - 1)
    y1 = np.clip(y1, 0, H - 1)
    
    # 取得四個角點的像素值
    I00 = image[y0, x0]
    I10 = image[y0, x1]
    I01 = image[y1, x0]
    I11 = image[y1, x1]
    
    # 雙線性插值
    output = (
        (1 - wx) * (1 - wy) * I00 +
        wx * (1 - wy) * I10 +
        (1 - wx) * wy * I01 +
        wx * wy * I11
    )
    
    # 超出邊界的設為 0
    output = np.where(valid, output, 0)
    
    return output

# 效能比較
import time

# For-loop 版本
start = time.time()
result_loop = rotate_bilinear(test_image, 30)
time_loop = time.time() - start

# 向量化版本
start = time.time()
result_vec = rotate_bilinear_vectorized(test_image, 30)
time_vec = time.time() - start

print(f"For-loop 版本：{time_loop:.4f} 秒")
print(f"向量化版本：  {time_vec:.4f} 秒")
print(f"加速比：      {time_loop / time_vec:.1f}x")

# 驗證結果一致
diff = np.abs(result_loop - result_vec)
print(f"\n結果差異（最大）：{diff.max():.6f}")

---

# Part 6: 仿射變換 (Affine Transform)

## 概念說明

**仿射變換** 是一種通用的線性變換，包含：
- 旋轉 (Rotation)
- 縮放 (Scaling)
- 剪切 (Shearing)
- 平移 (Translation)

### 數學表示（齊次座標）：

$$
\begin{bmatrix} x' \\ y' \\ 1 \end{bmatrix} = 
\begin{bmatrix} 
a & b & t_x \\
c & d & t_y \\
0 & 0 & 1 
\end{bmatrix}
\begin{bmatrix} x \\ y \\ 1 \end{bmatrix}
$$

### 各種變換的矩陣：

**縮放**：
$$
S = \begin{bmatrix} s_x & 0 & 0 \\ 0 & s_y & 0 \\ 0 & 0 & 1 \end{bmatrix}
$$

**旋轉**：
$$
R = \begin{bmatrix} \cos\theta & -\sin\theta & 0 \\ \sin\theta & \cos\theta & 0 \\ 0 & 0 & 1 \end{bmatrix}
$$

**平移**：
$$
T = \begin{bmatrix} 1 & 0 & t_x \\ 0 & 1 & t_y \\ 0 & 0 & 1 \end{bmatrix}
$$

**剪切**：
$$
Sh = \begin{bmatrix} 1 & sh_x & 0 \\ sh_y & 1 & 0 \\ 0 & 0 & 1 \end{bmatrix}
$$

In [ ]:
def affine_matrix_scale(sx, sy):
    """縮放矩陣"""
    return np.array([
        [sx, 0, 0],
        [0, sy, 0],
        [0, 0, 1]
    ], dtype=np.float64)


def affine_matrix_rotate(angle_degrees):
    """旋轉矩陣（3x3 齊次座標）"""
    theta = np.radians(angle_degrees)
    cos_t = np.cos(theta)
    sin_t = np.sin(theta)
    return np.array([
        [cos_t, -sin_t, 0],
        [sin_t, cos_t, 0],
        [0, 0, 1]
    ], dtype=np.float64)


def affine_matrix_translate(tx, ty):
    """平移矩陣"""
    return np.array([
        [1, 0, tx],
        [0, 1, ty],
        [0, 0, 1]
    ], dtype=np.float64)


def affine_matrix_shear(shx, shy):
    """剪切矩陣"""
    return np.array([
        [1, shx, 0],
        [shy, 1, 0],
        [0, 0, 1]
    ], dtype=np.float64)


def apply_affine_transform(image, M, output_shape=None):
    """
    對影像套用仿射變換。
    
    Parameters
    ----------
    image : np.ndarray, shape (H, W)
        輸入影像
    M : np.ndarray, shape (3, 3)
        仿射變換矩陣
    output_shape : tuple, optional
        輸出影像大小，預設與輸入相同
    
    Returns
    -------
    output : np.ndarray
        變換後的影像
    """
    H, W = image.shape
    if output_shape is None:
        out_H, out_W = H, W
    else:
        out_H, out_W = output_shape
    
    # 計算逆矩陣（用於反向映射）
    M_inv = np.linalg.inv(M)
    
    # 建立輸出座標網格
    y_out, x_out = np.mgrid[0:out_H, 0:out_W]
    ones = np.ones_like(x_out)
    
    # 齊次座標
    coords_out = np.stack([x_out, y_out, ones], axis=0)  # shape (3, H, W)
    coords_out = coords_out.reshape(3, -1)  # shape (3, H*W)
    
    # 反向變換
    coords_in = M_inv @ coords_out  # shape (3, H*W)
    x_orig = coords_in[0].reshape(out_H, out_W)
    y_orig = coords_in[1].reshape(out_H, out_W)
    
    # 雙線性插值（與之前相同的邏輯）
    x0 = np.floor(x_orig).astype(int)
    y0 = np.floor(y_orig).astype(int)
    x1 = x0 + 1
    y1 = y0 + 1
    
    wx = x_orig - x0
    wy = y_orig - y0
    
    valid = (x0 >= 0) & (x1 < W) & (y0 >= 0) & (y1 < H)
    
    x0 = np.clip(x0, 0, W - 1)
    x1 = np.clip(x1, 0, W - 1)
    y0 = np.clip(y0, 0, H - 1)
    y1 = np.clip(y1, 0, H - 1)
    
    I00 = image[y0, x0]
    I10 = image[y0, x1]
    I01 = image[y1, x0]
    I11 = image[y1, x1]
    
    output = (
        (1 - wx) * (1 - wy) * I00 +
        wx * (1 - wy) * I10 +
        (1 - wx) * wy * I01 +
        wx * wy * I11
    )
    
    output = np.where(valid, output, 0)
    
    return output

# 測試各種變換
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# 原圖
axes[0, 0].imshow(test_image, cmap='gray', vmin=0, vmax=255)
axes[0, 0].set_title('Original')

# 縮放 1.5x
M_scale = affine_matrix_scale(1.5, 1.5)
scaled = apply_affine_transform(test_image, M_scale)
axes[0, 1].imshow(scaled, cmap='gray', vmin=0, vmax=255)
axes[0, 1].set_title('Scale 1.5x')

# 縮放 0.5x
M_scale_small = affine_matrix_scale(0.5, 0.5)
scaled_small = apply_affine_transform(test_image, M_scale_small)
axes[0, 2].imshow(scaled_small, cmap='gray', vmin=0, vmax=255)
axes[0, 2].set_title('Scale 0.5x')

# 剪切 X
M_shear = affine_matrix_shear(0.3, 0)
sheared = apply_affine_transform(test_image, M_shear)
axes[1, 0].imshow(sheared, cmap='gray', vmin=0, vmax=255)
axes[1, 0].set_title('Shear X')

# 剪切 Y
M_shear_y = affine_matrix_shear(0, 0.3)
sheared_y = apply_affine_transform(test_image, M_shear_y)
axes[1, 1].imshow(sheared_y, cmap='gray', vmin=0, vmax=255)
axes[1, 1].set_title('Shear Y')

# 組合變換：縮放 + 旋轉
H, W = test_image.shape
T1 = affine_matrix_translate(-W/2, -H/2)  # 移到中心
R = affine_matrix_rotate(30)
S = affine_matrix_scale(0.8, 0.8)
T2 = affine_matrix_translate(W/2, H/2)   # 移回來
M_combo = T2 @ R @ S @ T1  # 注意順序：從右到左
combo = apply_affine_transform(test_image, M_combo)
axes[1, 2].imshow(combo, cmap='gray', vmin=0, vmax=255)
axes[1, 2].set_title('Scale 0.8x + Rotate 30°')

for ax in axes.flat:
    ax.axis('off')

plt.tight_layout()
plt.show()

---

# Part 7: 練習題

## 練習 1: 實作影像翻轉

In [ ]:
def flip_horizontal(image):
    """
    水平翻轉影像（左右鏡像）。
    
    Parameters
    ----------
    image : np.ndarray
        輸入影像
    
    Returns
    -------
    flipped : np.ndarray
        翻轉後的影像
    
    提示：使用仿射矩陣 [[-1, 0, W], [0, 1, 0], [0, 0, 1]]
    """
    H, W = image.shape
    M = np.array([
        [-1, 0, W - 1],
        [0, 1, 0],
        [0, 0, 1]
    ], dtype=np.float64)
    
    return apply_affine_transform(image, M)


def flip_vertical(image):
    """
    垂直翻轉影像（上下鏡像）。
    """
    H, W = image.shape
    M = np.array([
        [1, 0, 0],
        [0, -1, H - 1],
        [0, 0, 1]
    ], dtype=np.float64)
    
    return apply_affine_transform(image, M)


def flip_both(image):
    """
    同時水平和垂直翻轉（等同於旋轉 180°）。
    """
    H, W = image.shape
    M = np.array([
        [-1, 0, W - 1],
        [0, -1, H - 1],
        [0, 0, 1]
    ], dtype=np.float64)
    
    return apply_affine_transform(image, M)


# 測試翻轉
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

axes[0].imshow(test_image, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Original')

axes[1].imshow(flip_horizontal(test_image), cmap='gray', vmin=0, vmax=255)
axes[1].set_title('Horizontal Flip')

axes[2].imshow(flip_vertical(test_image), cmap='gray', vmin=0, vmax=255)
axes[2].set_title('Vertical Flip')

axes[3].imshow(flip_both(test_image), cmap='gray', vmin=0, vmax=255)
axes[3].set_title('Both Flip (= Rotate 180°)')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

## 練習 2: 實作影像縮放

In [ ]:
def resize_image(image, new_height, new_width, interpolation='bilinear'):
    """
    縮放影像到指定大小。
    
    Parameters
    ----------
    image : np.ndarray
        輸入影像
    new_height, new_width : int
        目標大小
    interpolation : str
        'nearest' 或 'bilinear'
    
    Returns
    -------
    resized : np.ndarray
        縮放後的影像
    """
    H, W = image.shape
    
    # 計算縮放比例
    scale_y = H / new_height
    scale_x = W / new_width
    
    # 建立輸出座標
    y_out = np.arange(new_height)
    x_out = np.arange(new_width)
    x_out, y_out = np.meshgrid(x_out, y_out)
    
    # 對應到原始座標
    y_orig = y_out * scale_y
    x_orig = x_out * scale_x
    
    if interpolation == 'nearest':
        # 最近鄰
        y_idx = np.round(y_orig).astype(int)
        x_idx = np.round(x_orig).astype(int)
        y_idx = np.clip(y_idx, 0, H - 1)
        x_idx = np.clip(x_idx, 0, W - 1)
        return image[y_idx, x_idx]
    
    elif interpolation == 'bilinear':
        # 雙線性
        x0 = np.floor(x_orig).astype(int)
        y0 = np.floor(y_orig).astype(int)
        x1 = x0 + 1
        y1 = y0 + 1
        
        wx = x_orig - x0
        wy = y_orig - y0
        
        x0 = np.clip(x0, 0, W - 1)
        x1 = np.clip(x1, 0, W - 1)
        y0 = np.clip(y0, 0, H - 1)
        y1 = np.clip(y1, 0, H - 1)
        
        I00 = image[y0, x0]
        I10 = image[y0, x1]
        I01 = image[y1, x0]
        I11 = image[y1, x1]
        
        return (
            (1 - wx) * (1 - wy) * I00 +
            wx * (1 - wy) * I10 +
            (1 - wx) * wy * I01 +
            wx * wy * I11
        )


# 測試縮放
upscaled = resize_image(test_image, 200, 200, 'bilinear')
downscaled = resize_image(test_image, 50, 50, 'bilinear')
upscaled_nn = resize_image(test_image, 200, 200, 'nearest')

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

axes[0].imshow(test_image, cmap='gray', vmin=0, vmax=255)
axes[0].set_title(f'Original ({test_image.shape[0]}x{test_image.shape[1]})')

axes[1].imshow(upscaled, cmap='gray', vmin=0, vmax=255)
axes[1].set_title(f'Upscaled 2x Bilinear ({upscaled.shape[0]}x{upscaled.shape[1]})')

axes[2].imshow(upscaled_nn, cmap='gray', vmin=0, vmax=255)
axes[2].set_title(f'Upscaled 2x NN ({upscaled_nn.shape[0]}x{upscaled_nn.shape[1]})')

axes[3].imshow(downscaled, cmap='gray', vmin=0, vmax=255)
axes[3].set_title(f'Downscaled 0.5x ({downscaled.shape[0]}x{downscaled.shape[1]})')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

## 練習 3: 完整的幾何變換類別

In [ ]:
class GeometricTransforms:
    """
    幾何變換工具類別。
    """
    
    @staticmethod
    def rotation_matrix(angle_degrees):
        """建立 3x3 旋轉矩陣"""
        theta = np.radians(angle_degrees)
        return np.array([
            [np.cos(theta), -np.sin(theta), 0],
            [np.sin(theta), np.cos(theta), 0],
            [0, 0, 1]
        ])
    
    @staticmethod
    def scale_matrix(sx, sy):
        """建立縮放矩陣"""
        return np.array([[sx, 0, 0], [0, sy, 0], [0, 0, 1]])
    
    @staticmethod
    def translate_matrix(tx, ty):
        """建立平移矩陣"""
        return np.array([[1, 0, tx], [0, 1, ty], [0, 0, 1]])
    
    @staticmethod
    def shear_matrix(shx, shy):
        """建立剪切矩陣"""
        return np.array([[1, shx, 0], [shy, 1, 0], [0, 0, 1]])
    
    @classmethod
    def rotate(cls, image, angle, center=None, interpolation='bilinear'):
        """旋轉影像"""
        H, W = image.shape
        if center is None:
            center = (W / 2, H / 2)
        
        cx, cy = center
        T1 = cls.translate_matrix(-cx, -cy)
        R = cls.rotation_matrix(angle)
        T2 = cls.translate_matrix(cx, cy)
        M = T2 @ R @ T1
        
        return apply_affine_transform(image, M)
    
    @classmethod
    def scale(cls, image, sx, sy, center=None):
        """縮放影像"""
        H, W = image.shape
        if center is None:
            center = (W / 2, H / 2)
        
        cx, cy = center
        T1 = cls.translate_matrix(-cx, -cy)
        S = cls.scale_matrix(sx, sy)
        T2 = cls.translate_matrix(cx, cy)
        M = T2 @ S @ T1
        
        return apply_affine_transform(image, M)
    
    @classmethod
    def shear(cls, image, shx=0, shy=0):
        """剪切影像"""
        M = cls.shear_matrix(shx, shy)
        return apply_affine_transform(image, M)
    
    @classmethod
    def flip(cls, image, horizontal=False, vertical=False):
        """翻轉影像"""
        H, W = image.shape
        sx = -1 if horizontal else 1
        sy = -1 if vertical else 1
        tx = W - 1 if horizontal else 0
        ty = H - 1 if vertical else 0
        
        M = np.array([[sx, 0, tx], [0, sy, ty], [0, 0, 1]], dtype=np.float64)
        return apply_affine_transform(image, M)

# 測試類別
print("=== GeometricTransforms 測試 ===")

gt = GeometricTransforms

rotated = gt.rotate(test_image, 45)
scaled = gt.scale(test_image, 0.7, 0.7)
sheared = gt.shear(test_image, shx=0.2)
flipped = gt.flip(test_image, horizontal=True)

fig, axes = plt.subplots(1, 5, figsize=(20, 4))

axes[0].imshow(test_image, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Original')

axes[1].imshow(rotated, cmap='gray', vmin=0, vmax=255)
axes[1].set_title('Rotate 45°')

axes[2].imshow(scaled, cmap='gray', vmin=0, vmax=255)
axes[2].set_title('Scale 0.7x')

axes[3].imshow(sheared, cmap='gray', vmin=0, vmax=255)
axes[3].set_title('Shear X=0.2')

axes[4].imshow(flipped, cmap='gray', vmin=0, vmax=255)
axes[4].set_title('Horizontal Flip')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

print("所有測試通過！")

---

# 總結

本 notebook 實作了以下幾何變換：

| 功能 | 描述 | 關鍵概念 |
|------|------|----------|
| 旋轉矩陣 | 2D 旋轉的數學表示 | cos/sin |
| 最近鄰插值 | 簡單但有鋸齒 | round() |
| 雙線性插值 | 平滑但稍模糊 | 四點加權平均 |
| 仿射變換 | 通用線性變換 | 3x3 齊次矩陣 |
| 縮放 | 改變影像大小 | scale matrix |
| 翻轉 | 鏡像變換 | 負縮放 |

## 關鍵概念

1. **Backward Mapping** 避免空洞問題
2. **雙線性插值** 提供比最近鄰更好的品質
3. **仿射變換** 可以組合多種變換
4. **向量化** 大幅提升效能

## 下一步

- 在 Module 2 中，我們會使用這些基礎概念來實作 **Canny 邊緣偵測**

In [ ]:
# 最終驗證
print("=== 函數驗證 ===")

test_funcs = [
    ("rotation_matrix(45)", lambda: rotation_matrix(45)),
    ("rotate_nearest_neighbor(image, 30)", lambda: rotate_nearest_neighbor(test_image, 30)),
    ("bilinear_interpolate(image, 50.5, 50.5)", lambda: bilinear_interpolate(test_image, 50.5, 50.5)),
    ("rotate_bilinear(image, 30)", lambda: rotate_bilinear(test_image, 30)),
    ("rotate_bilinear_vectorized(image, 30)", lambda: rotate_bilinear_vectorized(test_image, 30)),
    ("affine_matrix_scale(1.5, 1.5)", lambda: affine_matrix_scale(1.5, 1.5)),
    ("affine_matrix_rotate(30)", lambda: affine_matrix_rotate(30)),
    ("affine_matrix_translate(10, 10)", lambda: affine_matrix_translate(10, 10)),
    ("affine_matrix_shear(0.2, 0)", lambda: affine_matrix_shear(0.2, 0)),
    ("apply_affine_transform(image, M)", lambda: apply_affine_transform(test_image, affine_matrix_rotate(30))),
    ("flip_horizontal(image)", lambda: flip_horizontal(test_image)),
    ("flip_vertical(image)", lambda: flip_vertical(test_image)),
    ("resize_image(image, 50, 50)", lambda: resize_image(test_image, 50, 50)),
    ("GeometricTransforms.rotate(image, 45)", lambda: GeometricTransforms.rotate(test_image, 45)),
    ("GeometricTransforms.scale(image, 0.5, 0.5)", lambda: GeometricTransforms.scale(test_image, 0.5, 0.5)),
]

for name, func in test_funcs:
    try:
        result = func()
        print(f"✓ {name}")
    except Exception as e:
        print(f"✗ {name}: {e}")

print("\n所有函數驗證完成！")